In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

2023-10-25 21:59:10.853183: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_a = pd.read_parquet('A/train_targets.parquet')
X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')

In [4]:
train_a

,time,pv_measurement
0,2019-06-02 22:00:00,0.00
1,2019-06-02 23:00:00,0.00
2,2019-06-03 00:00:00,0.00
3,2019-06-03 01:00:00,0.00
4,2019-06-03 02:00:00,19.36
...,...,...
34080,2023-04-30 19:00:00,9.02
34081,2023-04-30 20:00:00,0.00
34082,2023-04-30 21:00:00,0.00
34083,2023-04-30 22:00:00,0.00


In [3]:
X_train_observed_a = X_train_observed_a.rename(columns={"date_forecast" : "time"})
X_train_observed_a


,time,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
0,2019-06-02 22:00:00,7.7,1.230,1744.900024,0.0,0.0,1744.900024,0.0,280.299988,0.0,0.0,0.0,0.0,98.699997,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1006.799988,0.0,0.0,994.200012,1000.299988,0.0,0.0,73.099998,1006.299988,NaN,0.0,0.0,-0.0,0.1,342.834015,-3.202000,0.0,285.899994,100.0,39640.101562,3.7,-3.6,-0.8,-0.0
1,2019-06-02 22:15:00,7.7,1.229,1734.000000,0.0,0.0,1734.000000,0.0,280.299988,0.0,0.0,0.0,0.0,99.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1006.500000,0.0,0.0,993.900024,999.900024,0.0,0.0,72.199997,1006.000000,NaN,0.0,0.0,-0.0,0.2,346.294006,-3.650000,0.0,286.100006,100.0,40123.898438,3.6,-3.6,-0.6,-0.0
2,2019-06-02 22:30:00,7.7,1.228,1723.500000,0.0,0.0,1723.500000,0.0,280.299988,0.0,0.0,0.0,0.0,99.199997,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1006.099976,0.0,0.0,993.599976,999.599976,0.0,0.0,71.199997,1005.599976,NaN,0.0,0.0,-0.0,0.2,349.768005,-3.998000,0.0,286.299988,100.0,40628.300781,3.6,-3.6,-0.4,-0.0
3,2019-06-02 22:45:00,7.7,1.226,1713.400024,0.0,0.0,1713.400024,0.0,280.299988,0.0,0.0,0.0,0.0,99.400002,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1005.799988,0.0,0.0,993.299988,999.299988,0.0,0.0,70.199997,1005.299988,NaN,0.0,0.0,-0.0,0.2,353.251007,-4.247000,0.0,286.600006,100.0,41153.601562,3.5,-3.5,-0.2,-0.0
4,2019-06-02 23:00:00,7.7,1.225,1703.599976,0.0,0.0,1703.599976,0.0,280.299988,0.0,0.0,0.0,0.0,99.599998,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1005.500000,0.0,0.0,993.000000,999.000000,0.0,0.0,69.199997,1005.000000,NaN,0.0,0.0,-0.0,0.2,356.742004,-4.393000,0.0,286.799988,100.0,41699.898438,3.5,-3.5,0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118664,2022-10-21 00:00:00,5.0,1.292,NaN,0.0,0.0,NaN,0.0,273.700012,0.0,0.0,0.0,0.0,2.800000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1018.400024,0.0,0.0,1005.500000,1011.799988,0.0,0.0,82.699997,1018.099976,NaN,0.0,0.0,-0.0,0.0,16.612000,-36.306000,0.0,276.899994,2.8,25791.000000,1.3,-0.4,1.2,-0.0
118665,2022-10-21 00:15:00,5.0,1.292,NaN,0.0,0.0,NaN,0.0,273.700012,0.0,0.0,0.0,0.0,2.300000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1018.299988,0.0,0.0,1005.400024,1011.700012,0.0,0.0,83.099998,1018.000000,NaN,0.0,0.0,-0.0,0.0,21.132999,-35.762001,0.0,276.799988,2.3,28560.599609,1.4,-0.6,1.2,-0.0
118666,2022-10-21 00:30:00,5.0,1.292,NaN,0.0,0.0,NaN,0.0,273.700012,0.0,0.0,0.0,0.0,1.700000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1018.200012,0.0,0.0,1005.299988,1011.599976,0.0,0.0,83.400002,1017.900024,NaN,0.0,0.0,-0.0,0.0,25.594000,-35.095001,0.0,276.600006,1.7,31317.800781,1.4,-0.8,1.2,-0.0
118667,2022-10-21 00:45:00,5.0,1.292,NaN,0.0,0.0,NaN,0.0,273.600006,0.0,0.0,0.0,0.0,1.100000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1018.000000,0.0,0.0,1005.099976,1011.400024,0.0,0.0,83.800003,1017.700012,NaN,0.0,0.0,-0.0,0.0,29.988001,-34.311001,0.0,276.500000,1.1,34062.601562,1.5,-1.0,1.1,-0.0


In [4]:
trainset = pd.merge(X_train_observed_a, train_a, how="inner", on=["time"])
trainsetX = trainset.iloc[:,1:-1]
trainsetY = trainset.iloc[:,-1]
trainsetX = trainsetX.fillna(method='ffill')
trainsetX = trainsetX.fillna(method='bfill')
trainset
#print(trainsetX[].isNull())

/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_39521/3750709821.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  trainsetX = trainsetX.fillna(method='ffill')
/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_39521/3750709821.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  trainsetX = trainsetX.fillna(method='bfill')


,time,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,pv_measurement
0,2019-06-02 22:00:00,7.7,1.230,1744.900024,0.000000,0.0,1744.900024,0.0,280.299988,0.0,0.000000,0.0,0.0,98.699997,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1006.799988,0.0,0.0,994.200012,1000.299988,0.0,0.0,73.099998,1006.299988,NaN,0.0,0.0,-0.0,0.1,342.834015,-3.202000,0.0,285.899994,100.000000,39640.101562,3.7,-3.6,-0.8,-0.0,0.00
1,2019-06-02 23:00:00,7.7,1.225,1703.599976,0.000000,0.0,1703.599976,0.0,280.299988,0.0,0.000000,0.0,0.0,99.599998,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1005.500000,0.0,0.0,993.000000,999.000000,0.0,0.0,69.199997,1005.000000,NaN,0.0,0.0,-0.0,0.2,356.742004,-4.393000,0.0,286.799988,100.000000,41699.898438,3.5,-3.5,0.0,-0.0,0.00
2,2019-06-03 00:00:00,7.7,1.221,1668.099976,0.000000,0.0,1668.099976,0.0,280.200012,0.0,0.000000,0.0,0.0,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1004.599976,0.0,0.0,992.099976,998.099976,0.0,0.1,66.000000,1004.099976,NaN,0.0,0.0,-0.0,0.2,9.741000,-3.910000,0.0,287.000000,100.000000,20473.000000,3.2,-3.1,0.7,-0.0,0.00
3,2019-06-03 01:00:00,8.2,1.218,1388.400024,0.000000,0.0,1388.400024,0.0,281.299988,0.0,0.000000,0.0,0.0,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1004.400024,0.0,0.0,991.799988,997.799988,0.0,0.2,71.099998,1003.799988,NaN,0.0,0.0,-0.0,0.7,23.541000,-1.986000,0.0,286.899994,100.000000,2104.600098,2.8,-2.7,0.8,-0.0,0.00
4,2019-06-03 02:00:00,8.8,1.219,1108.500000,6546.899902,9.8,1108.500000,0.0,282.299988,4.3,7743.299805,0.0,0.0,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1003.400024,0.0,0.0,990.900024,996.900024,0.0,0.1,78.500000,1002.900024,NaN,0.0,0.0,-0.0,0.3,37.040001,1.401000,0.0,286.500000,100.000000,2681.600098,2.7,-2.5,1.0,-0.0,19.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29662,2022-10-20 20:00:00,5.9,1.279,1291.500000,0.000000,0.0,1302.500000,0.0,276.000000,0.0,0.000000,0.0,0.0,48.900002,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1019.599976,0.0,0.0,1006.700012,1012.900024,0.0,0.0,78.599998,1019.200012,NaN,0.0,0.0,-0.0,0.0,307.263000,-27.830000,0.0,278.399994,59.400002,41022.398438,2.0,0.6,1.9,-0.0,0.00
29663,2022-10-20 21:00:00,5.7,1.283,NaN,0.000000,0.0,1009.299988,0.0,275.500000,0.0,0.000000,0.0,0.0,18.700001,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1019.299988,0.0,0.0,1006.500000,1012.700012,0.0,0.0,78.699997,1019.000000,NaN,0.0,0.0,-0.0,0.0,323.514008,-32.546001,0.0,278.000000,23.799999,38581.898438,1.8,0.3,1.8,-0.0,0.00
29664,2022-10-20 22:00:00,5.4,1.286,NaN,0.000000,0.0,149.500000,0.0,274.700012,0.0,0.000000,0.0,0.0,12.200000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1019.099976,0.0,0.0,1006.200012,1012.400024,0.0,0.0,79.800003,1018.700012,NaN,0.0,0.0,-0.0,0.0,341.010986,-35.674000,0.0,277.600006,12.200000,39697.699219,1.5,0.1,1.5,-0.0,0.00
29665,2022-10-20 23:00:00,5.3,1.289,NaN,0.000000,0.0,NaN,0.0,274.399994,0.0,0.000000,0.0,0.0,5.500000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1018.799988,0.0,0.0,1006.000000,1012.200012,0.0,0.0,80.300003,1018.500000,NaN,0.0,0.0,-0.0,0.0,359.334991,-36.821999,0.0,277.299988,5.500000,28090.099609,1.3,-0.0,1.3,-0.0,0.00


In [9]:

normalizer = tf.keras.layers.Normalization()
normalizer.adapt(trainsetX)

model = tf.keras.models.Sequential([
  normalizer,
  tf.keras.layers.Dense(45, activation='relu'),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(1)
])
model.compile(
  loss='mean_absolute_error'
  )
#print(model.predict(trainsetX))

In [10]:
history = model.fit(trainsetX, trainsetY, batch_size=1000, epochs=2000)

Epoch 1/2000
30/30 [==============================] - 2s 6ms/step - loss: 668.4491
Epoch 2/2000
30/30 [==============================] - 0s 6ms/step - loss: 623.1132
Epoch 3/2000
30/30 [==============================] - 0s 5ms/step - loss: 505.9024
Epoch 4/2000
30/30 [==============================] - 0s 5ms/step - loss: 365.5135
Epoch 5/2000
30/30 [==============================] - 0s 6ms/step - loss: 294.9932
Epoch 6/2000
30/30 [==============================] - 0s 6ms/step - loss: 264.5290
Epoch 7/2000
30/30 [==============================] - 0s 6ms/step - loss: 247.4028
Epoch 8/2000
30/30 [==============================] - 0s 5ms/step - loss: 239.1198
Epoch 9/2000
30/30 [==============================] - 0s 6ms/step - loss: 235.5867
Epoch 10/2000
30/30 [==============================] - 0s 7ms/step - loss: 233.3434
Epoch 11/2000
30/30 [==============================] - 0s 5ms/step - loss: 231.9709
Epoch 12/2000
30/30 [==============================] - 0s 5ms/step - loss: 231.0945
E

KeyboardInterrupt: 

In [67]:
preds = model.predict(trainsetX)

928/928 [==============================] - 2s 2ms/step


In [68]:
compare = pd.concat([trainsetY, pd.DataFrame(preds)], axis=1)
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 5,
                       'display.float_format', lambda x: '%.3f' % x,
                       ):
    print(compare)

       pv_measurement        0
0               0.000    7.126
1               0.000    7.126
2               0.000    7.126
3               0.000    7.126
4              19.360    7.126
5             251.020  120.138
6             263.780   55.973
7             522.720  608.182
8             904.420 1002.422
9            1238.820  810.026
10           2189.880 2135.098
11           3047.220 2886.791
12           2163.260 1884.569
13           2686.640 3067.411
14           3175.920 2854.539
15           2730.860 2130.327
16           2093.960 1871.876
17           2774.200 1490.150
18           1833.480 1317.466
19           1057.540  726.142
20            340.120  379.418
21             48.400  118.402
22             12.540    7.126
23              0.000    7.126
24              0.000    7.126
25              0.000    7.126
26              0.000    7.126
27              6.380    7.126
28            117.040  109.457
29            305.580  242.794
30            805.420  428.762
31      

In [11]:
X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('A/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('A/X_test_estimated.parquet')

X_test_estimated_a = X_test_estimated_a.rename(columns={"date_forecast" : "time"})
X_test_estimated_b = X_test_estimated_b.rename(columns={"date_forecast" : "time"})
X_test_estimated_c = X_test_estimated_c.rename(columns={"date_forecast" : "time"})
X_test_estimated_a["location"] = "A"
X_test_estimated_b["location"] = "B"
X_test_estimated_c["location"] = "C"
X_test_estimated = pd.concat([X_test_estimated_a, X_test_estimated_b,X_test_estimated_c])


parse_dates = ['time']
X_test_targets = pd.read_csv("test.csv", parse_dates=parse_dates)
X_test_targets.dtypes

X_test = pd.merge(X_test_estimated, X_test_targets, on=["time", "location"], how="right")
X_test = X_test.fillna(method='ffill')
X_test = X_test.fillna(method='bfill')
X_test = X_test.fillna(value=0)
X_test = X_test.iloc[:,2:-3]
X_test
test_preds = pd.read_csv("sample_submission.csv")
test_preds["prediction"] = model.predict(X_test)
test_preds.to_csv("testtesttest.csv", index=False)

/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_39521/2803706593.py:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_test = X_test.fillna(method='ffill')
/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_39521/2803706593.py:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_test = X_test.fillna(method='bfill')


68/68 [==============================] - 0s 2ms/step
